<a href="https://colab.research.google.com/github/dellaanima/AIFFEL_QUEST/blob/master/finetune-hfmodels/00_klue_bert_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.3 MB/s eta 0:00:00


In [ ]:
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm -rf ~/.cache/matplotlib

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

Refenrece : https://wikidocs.net/119990

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#  TPU 초기화
import tensorflow as tf
import os

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])

tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)



In [ ]:
strategy = tf.distribute.TPUStrategy(resolver)

In [ ]:
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re


In [ ]:

data = pd.read_csv('/content/drive/MyDrive/EDA_TRAIN_FINAL_v1.csv')
data.head(20)

,Unnamed: 0,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
5,5,직장 내 괴롭힘 대화,나 이틀뒤에 가나다 음식점 예약좀 해줘. 저녁7시로.\n가나다 음식점이요.?\n응....
6,6,기타 괴롭힘 대화,35번 손님 아이스커피 두잔나왔습니다\n아이스커피? \n네 맛있게드세요\n저기요 아...
7,7,기타 괴롭힘 대화,경비아저씨 내차에 경고장 붙였어요?\n내가 여기 몇년째 사는데 아직도 내차인줄 모르...
8,8,기타 괴롭힘 대화,이거 할인 된다면서요\n 확인해보겠습니다\n 아까 저 시람은 할인 해주던데\n 네 ...
9,9,갈취 대화,자네 세금계산서 보내준 거 메일이랑 수치가 다르던데?\n그게.\n얼마나 해먹었어? ...


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4342 entries, 0 to 4341
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    4342 non-null   int64 
 1   class         4342 non-null   object
 2   conversation  4342 non-null   object
dtypes: int64(1), object(2)
memory usage: 101.9+ KB


In [ ]:
data.duplicated().sum()

0

In [ ]:
data['class'].value_counts()

협박 대화          1239
기타 괴롭힘 대화      1094
갈취 대화          1030
직장 내 괴롭힘 대화     979
Name: class, dtype: int64

In [ ]:
def preprocess_sentence(sentence):

    sentence = sentence.lower().strip()
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z?.!가-힣ㄱ-ㅎㅏ-ㅣ]+", " ", sentence)
    sentence = sentence.strip()

    return sentence


In [ ]:

data['conversation'] = data['conversation'].apply(lambda x : preprocess_sentence(x))
data.head(20)


Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(data['class'])
num_labels = len(label_encoder.classes_)

data['class'] = np.asarray(label_encoder.transform(data['class']), dtype=np.int32)
data.head(10)


,Unnamed: 0,class,conversation
0,0,3,지금 너 스스로를 죽여달라고 애원하는 것인가 ? 아닙니다 . 죄송합니다 . 죽을 거...
1,1,3,길동경찰서입니다 . 시 분 마트에 폭발물을 설치할거다 . 네 ? 똑바로 들어 한번만...
2,2,1,너 되게 귀여운거 알지 ? 나보다 작은 남자는 첨봤어 . 그만해 . 니들 놀리는거 ...
3,3,0,어이 거기 예 ? ? 너 말이야 너 . 이리 오라고 무슨 일 . 너 옷 좋아보인다 ...
4,4,0,저기요 혹시 날이 너무 뜨겁잖아요 ? 저희 회사에서 이 선크림 파는데 한 번 손등에...
5,5,2,나 이틀뒤에 가나다 음식점 예약좀 해줘 . 저녁 시로 . 가나다 음식점이요 . ? ...
6,6,1,번 손님 아이스커피 두잔나왔습니다 아이스커피 ? 네 맛있게드세요 저기요 아가씨 나는...
7,7,1,경비아저씨 내차에 경고장 붙였어요 ? 내가 여기 몇년째 사는데 아직도 내차인줄 모르...
8,8,1,이거 할인 된다면서요 확인해보겠습니다 아까 저 시람은 할인 해주던데 네 저 손님은 ...
9,9,0,자네 세금계산서 보내준 거 메일이랑 수치가 다르던데 ? 그게 . 얼마나 해먹었어 ?...


In [ ]:
data['class'].value_counts()

3    1239
1    1094
0    1030
2     979
Name: class, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# conversation : x , label : y
conversation = data['conversation'].tolist()
label = data['class'].tolist()

# train, val 분리 (데이터 0.2)
train_x, val_x, train_y, val_y = train_test_split(conversation, label, random_state=42, test_size=0.2)

In [ ]:
len(train_x), len(val_x)

(3473, 869)

In [ ]:
# trainset-set
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_y
))

# validation-set
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_y
))

In [ ]:
# trainset-set
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_y
))

# validation-set
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_y
))

In [ ]:
from transformers import TFBertForSequenceClassification


with strategy.scope():
  num_labels = num_labels
  model = TFBertForSequenceClassification.from_pretrained(HUGGINGFACE_MODEL_PATH, num_labels=num_labels, from_pt=True)
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.summary()

Model: "tf_bert_for_sequence_classification_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  110617344 
                                                                 
 dropout_189 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  3076      
                                                                 
Total params: 110,620,420
Trainable params: 110,620,420
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

callback_earlystop = EarlyStopping(
    monitor="val_accuracy",
    min_delta=0.001, # acc가 최소한 얼마만큼의 상승이 있어야하는지 정해줌.
    patience=3)

history = model.fit(
    train_dataset.shuffle(len(train_dataset)).batch(16), epochs=10, batch_size=16,
    validation_data=val_dataset.shuffle(len(val_dataset)).batch(16),
    callbacks = [callback_earlystop]
)


Epoch 1/10
218/218 [==============================] - 149s 197ms/step - loss: 0.4683 - accuracy: 0.8367 - val_loss: 0.3340 - val_accuracy: 0.8815
Epoch 2/10
218/218 [==============================] - 31s 140ms/step - loss: 0.1876 - accuracy: 0.9401 - val_loss: 0.3060 - val_accuracy: 0.9010
Epoch 3/10
218/218 [==============================] - 31s 140ms/step - loss: 0.0827 - accuracy: 0.9758 - val_loss: 0.3597 - val_accuracy: 0.9022
Epoch 4/10
218/218 [==============================] - 31s 141ms/step - loss: 0.0611 - accuracy: 0.9821 - val_loss: 0.4056 - val_accuracy: 0.9010
Epoch 5/10
218/218 [==============================] - 31s 144ms/step - loss: 0.0543 - accuracy: 0.9824 - val_loss: 0.4852 - val_accuracy: 0.9022
Epoch 6/10
218/218 [==============================] - 31s 144ms/step - loss: 0.0346 - accuracy: 0.9891 - val_loss: 0.4428 - val_accuracy: 0.9022


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

callback_earlystop = EarlyStopping(
    monitor="val_accuracy",
    min_delta=0.001, # acc가 최소한 얼마만큼의 상승이 있어야하는지 정해줌.
    patience=3)

history = model.fit(
    train_dataset.shuffle(len(train_dataset)).batch(16), epochs=8, batch_size=16,
    validation_data=val_dataset.shuffle(len(val_dataset)).batch(16),
    callbacks = [callback_earlystop]
)


Epoch 1/8
218/218 [==============================] - 31s 143ms/step - loss: 0.0285 - accuracy: 0.9919 - val_loss: 0.4289 - val_accuracy: 0.9148
Epoch 2/8
218/218 [==============================] - 30s 139ms/step - loss: 0.0182 - accuracy: 0.9951 - val_loss: 0.4417 - val_accuracy: 0.9079
Epoch 3/8
218/218 [==============================] - 31s 141ms/step - loss: 0.0218 - accuracy: 0.9957 - val_loss: 0.4688 - val_accuracy: 0.8953
Epoch 4/8
218/218 [==============================] - 31s 140ms/step - loss: 0.0587 - accuracy: 0.9833 - val_loss: 0.4370 - val_accuracy: 0.8999


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

callback_earlystop = EarlyStopping(
    monitor="val_accuracy",
    min_delta=0.001, # acc가 최소한 얼마만큼의 상승이 있어야하는지 정해줌.
    patience=3)

history = model.fit(
    train_dataset.shuffle(len(train_dataset)).batch(16), epochs=6, batch_size=16,
    validation_data=val_dataset.shuffle(len(val_dataset)).batch(16),
    callbacks = [callback_earlystop]
)


Epoch 1/6
218/218 [==============================] - 33s 152ms/step - loss: 0.0236 - accuracy: 0.9931 - val_loss: 0.4316 - val_accuracy: 0.9114
Epoch 2/6
218/218 [==============================] - 31s 141ms/step - loss: 0.0091 - accuracy: 0.9983 - val_loss: 0.4295 - val_accuracy: 0.9148
Epoch 3/6
218/218 [==============================] - 31s 143ms/step - loss: 0.0216 - accuracy: 0.9928 - val_loss: 0.4864 - val_accuracy: 0.9045
Epoch 4/6
218/218 [==============================] - 31s 141ms/step - loss: 0.0481 - accuracy: 0.9868 - val_loss: 0.5302 - val_accuracy: 0.8964
Epoch 5/6
218/218 [==============================] - 31s 142ms/step - loss: 0.0130 - accuracy: 0.9974 - val_loss: 0.5146 - val_accuracy: 0.9079


In [ ]:


id2labels = model.config.id2label
model.config.id2label = {id : label_encoder.inverse_transform([int(re.sub('LABEL_', '', label))])[0]  for id, label in id2labels.items()}

label2ids = model.config.label2id
model.config.label2id = {label_encoder.inverse_transform([int(re.sub('LABEL_', '', label))])[0] : id   for id, label in id2labels.items()}



In [ ]:
import os
DIR_PATH = '/content/drive/MyDrive/dlton/model'
MODEL_NAME = 'klue-bert-base_5ep'
MODEL_SAVE_PATH = os.path.join(DIR_PATH, MODEL_NAME) # change this to your preferred location

if os.path.exists(MODEL_SAVE_PATH):
    print(f"{MODEL_SAVE_PATH} -- Folder already exists \n")
else:
    os.makedirs(MODEL_SAVE_PATH, exist_ok=True)
    print(f"{MODEL_SAVE_PATH} -- Folder create complete \n")

# save tokenizer, model
model.save_pretrained(MODEL_SAVE_PATH)
tokenizer.save_pretrained(MODEL_SAVE_PATH)

/content/drive/MyDrive/dlton/model/klue-bert-base_5ep -- Folder create complete 



('/content/drive/MyDrive/dlton/model/klue-bert-base_5ep/tokenizer_config.json',
 '/content/drive/MyDrive/dlton/model/klue-bert-base_5ep/special_tokens_map.json',
 '/content/drive/MyDrive/dlton/model/klue-bert-base_5ep/vocab.txt',
 '/content/drive/MyDrive/dlton/model/klue-bert-base_5ep/added_tokens.json')

In [ ]:
from transformers import TextClassificationPipeline

# Load Fine-tuning model
loaded_tokenizer = BertTokenizer.from_pretrained(MODEL_SAVE_PATH)
loaded_model = TFBertForSequenceClassification.from_pretrained(MODEL_SAVE_PATH)

text_classifier = TextClassificationPipeline(
    tokenizer=loaded_tokenizer,
    model=loaded_model,
    framework='tf',
    top_k=None
)

Some layers from the model checkpoint at /content/drive/MyDrive/dlton/model/klue-bert-base_5ep were not used when initializing TFBertForSequenceClassification: ['dropout_189']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/dlton/model/klue-bert-base_5ep.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [ ]:
# Load Test-set
test_file_path = '/content/drive/MyDrive/test (1).json'
with open(test_file_path, mode='rt', encoding='utf-8') as f:
    test_dataset = pd.read_json(f)
test_dataset

,t_000,t_001,t_002,t_004,t_005,t_006,t_007,t_009,t_010,t_012,...,t_489,t_490,t_492,t_493,t_494,t_495,t_496,t_497,t_498,t_499
text,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...,그러니까 빨리 말해. 선생님 제발 살려주십시오. 비밀번호 틀릴 때마다 손톱 하나씩...,그 사람 누구냐? 누구말하는거야? 다 알면서 모른척 하지마. 둘다 쏴버리기 전에...,야 저기야 닭꼬치 트럭왔다 응 그러네 그치? 너도 먹고 싶지? 어?나도? 그래 너 ...,야 너 2학년 김민석 맞지? 네 맞는데요. 혹시 누구신가요? 내가 누군지 궁금하면 ...,예전에 네가 나한테 했던 일 기억하지? 너도 당해봐 예전 일이라면 내가 사과할게. ...,김비서 지금 우리애 학교가서 집에다가 좀 데려다 주세요 사장님 그런 개인적인 일은 ...,...,너 어제 집에 바로 안 들어갔지? 어떻게 알았어? 내가 눈이 좋아 왜 묻는건데? 너...,철수야 말 들어야지 싫은데 철밥통 선생아 너 선생님한테 말버릇이 뭐야 야 내가 너말...,동생이쁘다. 나주라 안돼 우리동생건들지마. 왜 안되냐? 좀 달라고 니 목숨값으로 동...,혹시 나 오늘 2만원만 빌려주만 안돼? 응? 안돼. 나도 요즘 맨날 택시타느라 택시...,이봐 네? 누구세요? 입다물고 가진거 다내놔 경찰에 신고할거에요 할 수 있으면 해봐...,미나씨 휴가 결제 올리기 전에 저랑 상의하라고 말한거 기억해요? 네 합니다. 보고서...,교수님 제 논문에 제 이름이 없나요? 아 무슨 논문말이야? 지난 번 냈던 논문이...,야 너 네 저요? 그래 너 왜요 돈좀 줘봐 돈 없어요 돈이 왜 없어 지갑은 폼이...,야 너 빨리 안 뛰어와? 너 이 환자 제대로 봤어 안 봤어 어제 저녁부터 계속 보다...,엄마 저 그 돈 안해주시면 정말 큰일나요. 이유도 말하지 않고. 몇번째니 경민아....


In [ ]:
test_data = test_dataset.transpose()

In [ ]:
test_data

In [ ]:
predicted_label_list = []
predicted_score_list = []

for text in test_data['text']:
    # predict
    preds_list = text_classifier(text)[0][0]
    predicted_label_list.append(preds_list['label']) # label
    predicted_score_list.append(preds_list['score']) # score

In [ ]:
submission = test_data.assign(CLASS=predicted_label_list)
submission.rename_axis('file_name', inplace=True)
submission = submission.rename(columns={'CLASS':'class'})

In [ ]:
submission.drop(['text'], axis=1, inplace=True)

In [ ]:

class_mapping = {
'협박 대화': 0,
'갈취 대화': 1,
'직장 내 괴롭힘 대화': 2,
'기타 괴롭힘 대화': 3

}
submission['class'] = [class_mapping[label] for label in submission['class']]
submission['class'] = np.asarray(submission['class'], dtype=np.int32)

In [ ]:
submission.head(10)

,class
file_name,
t_000,1
t_001,2
t_002,2
t_004,3
t_005,0
t_006,0
t_007,3
t_009,1
t_010,0


In [ ]:
import pandas as pd


submission_file_path = '/content/drive/MyDrive/dlton/klue-bert-base_5epo_submission.csv'
submission.to_csv(submission_file_path, index=True)

